In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory. 
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
EMBEDDING_FILE = 'adfasfad'#'/home/kai/data/resources/glove/glove.840B.300d.txt' #glove.twitter.27B.200d.txt
PATH = '~/data/toxic/data/'
# train = pd.read_csv(PATH + 'train.csv')
# test = pd.read_csv(PATH + 'test.csv')
# train = pd.read_csv(PATH + 'cleaned_train.csv')
# test = pd.read_csv(PATH + 'cleaned_test.csv')
train = pd.read_csv(PATH + 'train_preprocessed.csv')
test = pd.read_csv(PATH + 'test_preprocessed.csv')

In [3]:
train.head(1)

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
0,explanation why the edits made under my userna...,0000997932d777bf,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0


In [4]:
def word2Vec(source):
    embed_size = 300
    if source.lower() == 'ft-common':
        file = '/home/kai/data/resources/FastText/crawl-300d-2M.vec'
    elif source.lower() == 'ft-wiki':
        file = '/home/kai/data/resources/FastText/wiki.en.vec'
    elif source.lower() == 'lex':
        file = '/home/kai/data/resources/lexvec/lexvec.commoncrawl.300d.W.pos.vectors'
    elif source.lower() == 'gl-common':
        file = '/home/kai/data/resources/glove/glove.840B.300d.txt'
    elif source.lower() == 'gl-twitter':
        file = '/home/kai/data/resources/glove/glove.twitter.27B.200d.txt'
        embed_size = 200
    def get_coefs(word,*arr): 
        try:
            return word, np.asarray(arr, dtype='float32') 
        except ValueError:
            return 'nnnnnnnaaaaaaa@@!',np.zeros(embed_size)
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(file, encoding='utf8'))
    return embeddings_index, embed_size

In [5]:
# embeddings_index_lex, embed_size = word2Vec('lex')
# embeddings_index_glc, embed_size = word2Vec('gl-common')
embeddings_index_glt, embed_size = word2Vec('gl-twitter')
# embeddings_index_ftc, embed_size = word2Vec('ft-common')
# embeddings_index_ftw, embed_size = word2Vec('ft-wiki')

In [8]:
type(embeddings_index_glt['didn'])

numpy.ndarray

In [9]:
comment_col = 'comment_text' # 'comment_text_cleaned' 

X_train = train[comment_col].str.lower().fillna('something') # something is a word of neutral sentiment
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

X_test = test[comment_col].str.lower().fillna('something')

In [10]:
max_features=100000
maxlen=150

tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(X_train)+list(X_test))
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)

In [11]:
sorted_word_count = sorted(tok.word_counts.items(), key=lambda x: x[1], reverse=True)

In [12]:
#sorted_word_count#[60000:]

In [13]:
# count = 0
# for w,w_cnt in sorted_word_count[:60000]:
#     if w in embeddings_index_glt:
#         count+=1
# print(count)
# print(count/60000)

In [14]:
# type(embeddings_index_glt['hesitates'])

In [15]:
import json as js
with open('/home/kai/data/kaggle/toxic/wl/models/RNN/rnn/dirty_word_dict.json', 'r') as file:
    bad_word_dict = js.load(file)

len(bad_word_dict)

228

In [16]:
# embeddings_index = embeddings_index_lex
# word_index = tok.word_index
# #prepare embedding matrix
# num_words = min(max_features, len(word_index) + 1)
# embedding_matrix = np.zeros((num_words, embed_size))
# not_found_word = {}
# for word, i in word_index.items():
#     if i >= max_features:
#         continue
#     try: 
#         embedding_vector = embeddings_index[word] # w2v_model['/en/'+ word] #w2v_model[word]#
#     except KeyError:
#         embedding_vector = embeddings_index.get(bad_word_dict.get(word), None)
# #         embedding_vector = None #np.zeros(embed_size)
#         if embedding_vector is None:
#             not_found_word[word] = tok.word_counts[word]#i
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
# print(len(not_found_word))

In [17]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [25]:
def get_embedding_matrix(embeddings_index, embed_size, max_features, tokenizer, bad_word_dict):
    word_index = tokenizer.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    embedding_matrix = np.zeros((num_words, embed_size))
    not_found_word = {}
    replaced_word = {}
    for word, i in word_index.items():
        if i >= max_features:
            continue
        try: 
            embedding_vector = embeddings_index[word] # w2v_model['/en/'+ word] #w2v_model[word]#
        except KeyError:
            replacement = bad_word_dict.get(word)
            embedding_vector = embeddings_index.get(replacement, None)
    #         embedding_vector = None #np.zeros(embed_size)
            if embedding_vector is None:
                not_found_word[word] = tokenizer.word_counts[word]#i
            else:
                if word not in replaced_word:
                    replaced_word[word] = replacement
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    print('{} words not found in embedding file'.format(len(not_found_word)))
    print('{} words are replaced:'.format(len(replaced_word)))
    print(replaced_word)
    return embedding_matrix, not_found_word, replaced_word

In [26]:
def get_model(maxlen, max_features, embedding_matrix, embed_size, output_size):
    sequence_input = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)

    conv1d_1 = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool_1 = GlobalAveragePooling1D()(conv1d_1) 
    max_pool_1 = GlobalMaxPooling1D()(conv1d_1)

    x = concatenate([avg_pool_1, max_pool_1])
    # x = Dense(128, activation='relu')(x)
    # x = Dropout(0.1)(x)
    preds = Dense(output_size, activation="sigmoid")(x)
    model = Model(sequence_input, preds)

    model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])
    
    return model

In [18]:
embeddings_index_pool = [
#     (embeddings_index_lex, 300, 'lex'),
#     (embeddings_index_glc, 300, 'glc'),
    (embeddings_index_glt, 200, 'glt')#,
#     (embeddings_index_ftc, 300, 'ftc'), 
#     (embeddings_index_ftw, 300, 'ftw')
]

In [19]:
batch_epoch_patience_pool = [
    (1024, 20, 5)
#     (1024, 10, 10),
#     (1024, 3, 3),
#     (1024, 3, 3),
#     (1024, 3, 3),
#     (512, 8, 8),
#     (512, 2, 2),
#     (512, 2, 2),
#     (512, 2, 2),
#     (128, 5, 5),
#     (128, 2, 2),
#     (128, 2, 2),
#     (128, 1, 1),
#     (32, 3, 3),
#     (32, 2, 2),
#     (32, 1, 1),
#     (32, 1, 1)
]

In [20]:
# train_index = np.load('train_index.npy')

# test_index = np.load('test_index.npy')

# print(train_index, test_index)

# real_train_index = [x for x in list(train_index) if x < len(train)]
# real_val_index = [x for x in list(test_index) if x < len(train)]

# assert len(real_train_index) + len(real_val_index) == len(train)

# X_tra, X_val = x_train[real_train_index, :], x_train[real_val_index, :]
# y_tra, y_val = y_train[real_train_index, :], y_train[real_val_index, :]

# print(X_tra.shape, X_val.shape, x_train.shape)
# print(y_tra.shape, y_val.shape, y_train.shape)

In [21]:
import gc
gc.collect()

243

In [22]:
x_train.shape, y_train.shape, x_test.shape

((159571, 150), (159571, 6), (153164, 150))

In [88]:
def adversarial_validation(x_train, y_train, x_test, embeddings_index, embed_size):
    """
    x_train, y_train, x_test: tokenized, indexed and padded data
    """
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import roc_auc_score
    
    xdat = np.vstack([x_train, x_test])
    print(xdat.shape)
    ydat = np.vstack([np.ones([len(x_train),1]), np.zeros([len(x_test),1])]) # train=>1, test=>0
    print(ydat.shape)
    
    y_val_oof_preds = np.ones([len(ydat), 1]) * 999 # we will check if there is still any 999 after the whole thing is done
    
    embedding_matrix, _, _ = get_embedding_matrix(embeddings_index, embed_size, max_features, tok, bad_word_dict)
    
    nfolds = 5
    xseed = 1001
    batch_size = 1024
    epochs = 10
    patience = 3
    # stratified split
    skf = StratifiedKFold(n_splits= nfolds, random_state= xseed)
    score_vec = np.zeros((nfolds,1))

    index_list = {}
    for (f, (train_index, test_index)) in enumerate(skf.split(xdat, ydat[:,0])):
        # split 
        X_tra, X_val = xdat[train_index], xdat[test_index]
        y_tra, y_val = ydat[train_index,0], ydat[test_index,0] 

        model = get_model(maxlen, max_features, embedding_matrix, embed_size, 1)
        
        run_name = 'cnn_nfolds_{}_fond_{}'.format(nfolds, f)
        print(run_name)
        filepath= './NewRnnModels/AdversarialValidation/' + run_name + '.hdf5'
        early = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        #ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
        callbacks_list = [checkpoint, early]

        model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)
        #Loading model weights
        model.load_weights(filepath)
        print('Predicting using the best model/epoch so far....')
        y_val_pred = model.predict(X_val,batch_size=1024,verbose=1)
        
        y_val_oof_preds[test_index] = y_val_pred

        roc_auc = roc_auc_score(y_val,y_val_pred)
        print(roc_auc)
        score_vec[f,:] = roc_auc
    
    return np.hstack([y_val_oof_preds, ydat]), score_vec

In [ ]:
y_val_oof_preds, score_vec = adversarial_validation(x_train, y_train, x_test, embeddings_index_glt, 200)

(312735, 150)
(312735, 1)
32208 words not found in embedding file
51 words are replaced:
{'busterd': 'bustard', 'britfags': 'faggot', 'fagsgod': 'faggot', 'dickus': 'dick', 'vagpenis': 'penis', 'bustards': 'bustard', 'pspsex': 'sex', 'homeostatic': 'homosexual', 'horsecock': 'cock', 'zdick': 'dick', 'dscc': 'dick', 'fuckerucker': 'fucker', "mother's": 'mother', 'niggors': 'nigger', 'semite': 'semitic', 'dickishness': 'dick', 'tdick': 'dick', "father's": 'father', 'terorists': 'terrorist', 'mothjer': 'mother', 'rapeing': 'rape', 'homopetersymonds': 'homo', 'cucks': 'cocks', 'burglaring': 'burglar', 'nonsesnse': 'nonsense', 'asslicker': 'ass', 'sexvbutt': 'sex', 'donkeysex': 'dick', 'fuock': 'fuck', 'niggetr': 'nigger', 'abuseful': 'abuse', 'faggt': 'faggot', 'fuckervckers': 'fuck', 'sockpuppetter': 'idiot', 'assraped': 'ass', 'headsdick': 'dick', 'peenus': 'penis', 'nonense': 'nonsense', 'diedres': 'crap', 'sockpuppetry': 'alias', 'noobish': 'noob', 'sockpuppet': 'alias', 'asspie': 'ass

Epoch 11/20
249856/250188 [============================>.] - ETA: 0s - loss: 0.5857 - acc: 0.6698
Epoch 00011: val_loss did not improve
250188/250188 [==============================] - 71s 284us/step - loss: 0.5857 - acc: 0.6697 - val_loss: 0.6194 - val_acc: 0.6353
Epoch 12/20
249856/250188 [============================>.] - ETA: 0s - loss: 0.5816 - acc: 0.6750
Epoch 00012: val_loss did not improve
250188/250188 [==============================] - 71s 283us/step - loss: 0.5816 - acc: 0.6750 - val_loss: 0.6214 - val_acc: 0.6331
Epoch 13/20
249856/250188 [============================>.] - ETA: 0s - loss: 0.5773 - acc: 0.6791
Epoch 00013: val_loss did not improve
250188/250188 [==============================] - 71s 285us/step - loss: 0.5773 - acc: 0.6790 - val_loss: 0.6236 - val_acc: 0.6318
Epoch 14/20
249856/250188 [============================>.] - ETA: 0s - loss: 0.5723 - acc: 0.6844
Epoch 00014: val_loss did not improve
250188/250188 [==============================] - 71s 286us/step - 

Epoch 12/20
249856/250188 [============================>.] - ETA: 0s - loss: 0.5820 - acc: 0.6734
Epoch 00012: val_loss did not improve
250188/250188 [==============================] - 71s 283us/step - loss: 0.5820 - acc: 0.6734 - val_loss: 0.6243 - val_acc: 0.6276
Predicting using the best model/epoch so far....
62547/62547 [==============================] - 5s 73us/step
0.6984662343198196
cnn_nfolds_5_fond_4
Train on 250189 samples, validate on 62546 samples
Epoch 1/20
249856/250189 [============================>.] - ETA: 0s - loss: 0.6579 - acc: 0.5884
Epoch 00001: val_loss improved from inf to 0.63724, saving model to ./NewRnnModels/AdversarialValidation/cnn_nfolds_5_fond_4.hdf5
250189/250189 [==============================] - 75s 299us/step - loss: 0.6579 - acc: 0.5884 - val_loss: 0.6372 - val_acc: 0.6075
Epoch 2/20
249856/250189 [============================>.] - ETA: 0s - loss: 0.6333 - acc: 0.6134
Epoch 00002: val_loss improved from 0.63724 to 0.62723, saving model to ./NewRnnM

In [92]:
y_val_oof_preds = y_oof_preds

In [93]:
np.save('y_val_oof_preds', y_val_oof_preds) # was saved with name: 'y_oof_preds'

In [89]:
def adversarial_train_val_split(x_train, y_train, y_val_oof_preds, val_size=0.1):
    try:
        ordered_indexes = np.load('pick_val_from_the_start.npy')
        print('ordered_indexes file loaded')
    except OSError:
        print('ordered_indexes file not found. computing...')
        print(y_val_oof_preds.shape)
        indexes = np.arange(len(y_val_oof_preds)).reshape(-1,1)
        print(indexes.shape)
        y_val_oof_preds_w_idx = np.hstack([indexes, y_val_oof_preds])
        # 0.0 0.7034685015678406 1.0
        # 1.0 0.994215190410614 1.0
        # 2.0 0.7640196084976196 1.0
        # 3.0 0.27269116044044495 1.0
        # 4.0 0.5975226759910583 1.0
        y_val_preds_dict = dict([(idx, y_val_pred) for idx, y_val_pred, y_val in y_val_oof_preds_w_idx if y_val == 1])
        print(len(y_val_preds_dict))
        print(list(y_val_preds_dict.items())[:3])
        sorted_y_val_preds = sorted(y_val_preds_dict.items(), key=lambda x:x[1])
        ordered_indexes = [int(idx) for idx in sorted(y_val_preds_dict, key=y_val_preds_dict.get)]
        np.save('pick_val_from_the_start', ordered_indexes)
        # ordered_indexes[:5] 
        # [(6833.0, 0.008921189233660698),
        #  (44761.0, 0.012650093995034695),
        #  (116571.0, 0.016914881765842438),
        #  (117959.0, 0.0190033670514822),
        #  (74465.0, 0.02322762832045555)......]

    split_point = int(len(ordered_indexes)*val_size)
    print('split point: {}. Before this point => val. After this point => train'.format(split_point))
    val_indexes = ordered_indexes[:split_point]
    train_indexes = ordered_indexes[split_point:]
    assert len(val_indexes) + len(train_indexes) == len(ordered_indexes) ==len(x_train)

    X_tra, X_val = x_train[train_indexes], x_train[val_indexes]
    y_tra, y_val = y_train[train_indexes], y_train[val_indexes]

    return X_tra, X_val, y_tra, y_val

In [ ]:
y_val_oof_preds = np.load('y_val_oof_preds.npy')

In [ ]:
X_tra, X_val, y_tra, y_val = adversarial_train_val_split(x_train, y_train, y_oof_preds)#y_val_oof_preds

In [94]:
for embeddings_index, embed_size, embedding_name in embeddings_index_pool:
    
    embedding_matrix, _, _ = get_embedding_matrix(embeddings_index, embed_size, max_features, tok, bad_word_dict)
    
    for batch_size, epochs, patience in batch_epoch_patience_pool:
        print(batch_size, epochs, patience)
        
        X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, test_size=0.1)#, random_state=233)
        #X_tra, X_val, y_tra, y_val = adversarial_train_val_split(x_train, y_train, y_val_oof_preds)

        model = get_model(maxlen, max_features, embedding_matrix, embed_size, 6)

        run_name = "rnn10_w_conv_{}_{}_w_new_dict".format(embedding_name, batch_size)
        print(run_name)

        filepath= './NewRnnModels/' + run_name + '.hdf5'
        
        try: 
            print('load model: ' + str(filepath))
            model.load_weights(filepath)
        except OSError: 
            print('no model found')
        
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        early = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
        ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
        callbacks_list = [ra_val,checkpoint, early]

        model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)
        #Loading model weights
        model.load_weights(filepath)
        print('Predicting using the best model/epoch so far....')
        y_pred = model.predict(x_test,batch_size=1024,verbose=1)

        y_pred.shape

        submission = pd.read_csv(PATH + 'sample_submission.csv')
        submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
        import time
        sub_id = int(time.time())
        print(sub_id)
        submission.to_csv('./NewRnnPreds/' + run_name + '_' + str(sub_id) + '.csv.gz', index=False, compression='gzip')

32208 words not found in embedding file
51 words are replaced:
{'busterd': 'bustard', 'britfags': 'faggot', 'fagsgod': 'faggot', 'dickus': 'dick', 'vagpenis': 'penis', 'bustards': 'bustard', 'pspsex': 'sex', 'homeostatic': 'homosexual', 'horsecock': 'cock', 'zdick': 'dick', 'dscc': 'dick', 'fuckerucker': 'fucker', "mother's": 'mother', 'niggors': 'nigger', 'semite': 'semitic', 'dickishness': 'dick', 'tdick': 'dick', "father's": 'father', 'terorists': 'terrorist', 'mothjer': 'mother', 'rapeing': 'rape', 'homopetersymonds': 'homo', 'cucks': 'cocks', 'burglaring': 'burglar', 'nonsesnse': 'nonsense', 'asslicker': 'ass', 'sexvbutt': 'sex', 'donkeysex': 'dick', 'fuock': 'fuck', 'niggetr': 'nigger', 'abuseful': 'abuse', 'faggt': 'faggot', 'fuckervckers': 'fuck', 'sockpuppetter': 'idiot', 'assraped': 'ass', 'headsdick': 'dick', 'peenus': 'penis', 'nonense': 'nonsense', 'diedres': 'crap', 'sockpuppetry': 'alias', 'noobish': 'noob', 'sockpuppet': 'alias', 'asspie': 'ass', 'sexbot': 'sex', 'maoth

In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_92 (InputLayer)           (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_92 (Embedding)        (None, 150, 300)     30000000    input_92[0][0]                   
__________________________________________________________________________________________________
spatial_dropout1d_92 (SpatialDr (None, 150, 300)     0           embedding_92[0][0]               
__________________________________________________________________________________________________
bidirectional_92 (Bidirectional (None, 150, 256)     329472      spatial_dropout1d_92[0][0]       
__________________________________________________________________________________________________
conv1d_92 

In [18]:
bad_word_dict = {
    "'ass": 'ass',
     "'shit": 'shit',
     "'stupid": 'stupid',
     'asse': 'ass',
     'asspie': 'ass',
     "bitch'": 'bitch',
     'bitchbot': 'bitch',
     'bith': 'bitch',
     'bithc': 'bitch',
     'bithces': 'bitch',
     'choked': 'choke',
     'cocain': 'cocaine',
     'cuck': 'cock',
     'cucks': 'cocks',
     'decease': 'deceased',
     'deth': 'death',
     'diedres': 'crap',
     'dik': 'dick',
     'donkeysex': 'dick',
     'faggt': 'faggot',
     "fool'": 'fool',
     "fuck'": 'fuck',
     'fuckerucker': 'fucker',
     'fuckn': 'fuck',
     'fuock': 'fuck',
     'gayy': 'gay',
     'headsdick': 'dick',
     'homopetersymonds': 'homo',
     'horsecock': 'cock',
     'maoth': 'mouth',
     "mother's": 'mother',
     'motherfuc': 'motherfucker',
     'mothjer': 'mother',
     'niggetr': 'nigger',
     'niggors': 'nigger',
     'nonense': 'nonsense',
     'nonsesnse': 'nonsense',
     'peenus': 'penis',
     'peni': 'penis',
     "penis'": 'penis',
     'pneis': 'penis',
     'pornn': 'porn',
     'semite': 'semitic',
     'sexe': 'sex',
     'sexsex': 'sex',
     'sexualit': 'sexuality',
     'sexuall': 'sexual',
     'sockpuppet': 'alias',
     'sockpuppetry': 'alias',
     'vagpenis': 'penis',
     'valentin': 'valentine',
     'youfuck': 'fuck',
     'zdick': 'dick'
}